In [14]:
import random
import torch
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [15]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

In [16]:
SEED = 1038893

fix_random(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

root = "../data/ml-25m"

Device: cpu


## Data Acquisition

In [17]:
def get_data_from_csv(file: str, nrows=None):
    if nrows:
        df = pd.read_csv(f"{root}/{file}", nrows=nrows)
    else:
        df = pd.read_csv(f"{root}/{file}")
    print(f"Loaded ml-25m data: {root}/{file}")

    return df

def get_ratings_from_csv(nrows=None):
    path = "ratings.csv"

    data = get_data_from_csv(path, nrows)
    # data = get_data_from_csv(path)

    data.drop("timestamp", axis=1, inplace=True)
    # todo: drop user id
    #data.drop("userId", axis=1, inplace=True)

    return data

def get_tag_relevances_from_csv(nrows=None):
    path = "genome-scores.csv"

    return get_data_from_csv(path, nrows)

In [18]:
genome_scores_path = "genome-scores.csv"
ratings_path = "ratings.csv"

In [19]:
ratings = get_ratings_from_csv()
print(ratings)

Loaded ml-25m data: ../data/ml-25m/ratings.csv
          userId  movieId  rating
0              1      296     5.0
1              1      306     3.5
2              1      307     5.0
3              1      665     5.0
4              1      899     3.5
...          ...      ...     ...
25000090  162541    50872     4.5
25000091  162541    55768     2.5
25000092  162541    56176     2.0
25000093  162541    58559     4.0
25000094  162541    63876     5.0

[25000095 rows x 3 columns]


In [20]:
genome_scores = get_tag_relevances_from_csv()
print(genome_scores)

Loaded ml-25m data: ../data/ml-25m/genome-scores.csv
          movieId  tagId  relevance
0               1      1    0.02875
1               1      2    0.02375
2               1      3    0.06250
3               1      4    0.07575
4               1      5    0.14075
...           ...    ...        ...
15584443   206499   1124    0.11000
15584444   206499   1125    0.04850
15584445   206499   1126    0.01325
15584446   206499   1127    0.14025
15584447   206499   1128    0.03350

[15584448 rows x 3 columns]


In [21]:
# films = get_data_from_csv(f"{root}/{ratings}")[]


# DATA VISUALIZATION


In [22]:
            # # Fill in missing values with zeros
            # X.fillna(0, inplace=True)


# FARE TEST CON AVG, STD_DEV

In [23]:
class SVM:
    def __init__(self, ratings, relevance, seed=42):
        # Pivot the relevance DataFrame to create a matrix of tag relevance scores for each movie
        relevance_matrix = relevance.pivot_table(index='movieId', columns='tagId', values='relevance', fill_value=0)

        # Compute the mean rating for each user
        mean_ratings = ratings.groupby('movieId', as_index=False)['rating'].mean()
        ratings = None

        # Merge the ratings and relevance data
        X = mean_ratings.merge(relevance_matrix, on='movieId')
        X.columns = X.columns.astype(str)

        y = np.round(X['rating'], decimals=0)
        X = X.drop('rating', axis=1)

        # Split the data into training and testing sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
        #print(self.X_train.shape, self.X_test.shape, self.y_train.shape, self.y_test.shape)

        # Initialize the SVM model
        self.svm = LinearSVR()

    # Training
    def train(self, num_epochs=10):
        svm = LinearSVR()
        for epoch in range(num_epochs):
            svm.fit(self.X_train, self.y_train)
            self.svm = svm
            print('training finished.')
            self.test()
        self.svm = svm


    def test(self):
            # Predict ratings for the test data
            y_pred = self.svm.predict(self.X_test)

            # Compute the mean squared error
            mse = mean_squared_error(self.y_test, y_pred)
            # Compute the score
            score = self.svm.score(self.X_test, y_pred)
            # Compute R^2
            r2 = r2_score(self.y_test, y_pred)
            # Compute the mean absolute error
            mae = mean_absolute_error(self.y_test, y_pred)

            print(f"Mean squared error: {mse}")
            print(f'Score: {score}')
            print(f"R^2: {r2}")
            print(f"Mean absolute error: {mae}")


In [24]:
bs = SVM(ratings, genome_scores, SEED)

In [25]:
bs.train(1)

training finished.
Mean squared error: 0.2151175698505215
Score: 1.0
R^2: 0.39889839087036216
Mean absolute error: 0.37007850047192103


c:\Users\ktm17\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [26]:
bs.test()

Mean squared error: 0.2151175698505215
Score: 1.0
R^2: 0.39889839087036216
Mean absolute error: 0.37007850047192103
